### Topic modeling
Extracting topics from the reddit messages 

##### Imports 

In [1]:
# Imports
from bertopic import BERTopic
import pandas as pd
import numpy as np
import nltk 

##### Data 

Loading the data 

In [2]:
# getting subset of data 
messages_df = pd.read_csv("reddit_22_51/messages.csv", sep="\t")
messages_df.head()

,id,user,text
0,j0s252k,HexagonOfVirtue,"im gonna find it just to check, it's not the ..."
1,j0s25h2,Teephex,According to you criticizing and being skeptic...
2,j0s25ht,1platesquat,Gotcha. Can you explain to me why your opinion...
3,j0s25l5,YouLostTheGame,"Euros, which some argue is actually harder tha..."
4,j0s25nr,HMID_Delenda_Est,You've been sounding more like PunishedSubSist...


Info on the data 

In [3]:
# info on data
column_list = messages_df.columns
shape = messages_df.shape

print("columns: ", column_list)
print("shape ", shape)

columns:  Index(['id', 'user', 'text'], dtype='object')
shape  (290898, 3)


**Data cleaning: stop word removal**

Stop word are very frequent words e.g. “the” and “a” that can impact the topics generated by the bertopic model due to their high frequency across most documents/texts - can be removed to get clearer more informative topics 

In [4]:
# METHOD 2: remove most frequent words found in the data 
def remove_stop_words(text, stop_words): 
    words = text.split() 
    # Use a list comprehension to remove stop words 
    filtered_words = [word for word in words if word.lower() not in stop_words] 
  
  # Join the filtered words back into a sentence 
    return ' '.join(filtered_words)

most_frequent_words = pd.Series(' '.join(messages_df['text']).lower().split()).value_counts()[:100].index.tolist()

messages_df_sw2 = messages_df.copy()
messages_df_sw2["text"] = messages_df_sw2["text"].apply(lambda row: remove_stop_words(row, most_frequent_words))

Convert data to list of strings (input needed by bertopic)

In [7]:
# convert to list of strings (input needed by bertopic model)
messages_list = messages_df_sw2["text"].astype(str).tolist()
len(messages_list)

290898

Get subset of data to work with 

In [8]:
# subset of data - 5000 rows
messages_subset = messages_list[:10000]
messages_subset

['im gonna find check, ai art right? triggered before rare',
 'According criticizing skeptical active politican mental disorder?',
 'Gotcha. explain opinion important me?',
 'Euros, argue actually harder WC',
 "You've sounding PunishedSubSister tbh",
 'hate race hate born',
 'communism abysmal failure ever single instances attempted? Yeah. Hans agree.',
 'made genuine effort try estimate cost moving affordable place?',
 "implemented rule saying couldn't link social media sites (Instagram, facebook, Mastodon, etc)",
 'Yes. Every single human earth eats tiny amounts poison regular basis. that? literally everything poisonous depending dose. Water itself poisonous drink enough',
 'set base couple days. Go read Volume 2 AFH 10-222 done.',
 'welcome opinion.',
 'Literally least worst Trump adjacent person sell to.',
 'Speech thought. Thought police.',
 'now happened school system.',
 'Thread OP: "so need landlords?" Me: "unironically yes" tbh long rental company doesn\'t monopoly certain are

##### Bertopic model 

In [9]:
topic_model = BERTopic()
# topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

In [10]:
# fitting the bertopic model 
topic_model_fitted = topic_model.fit(messages_subset)

KeyboardInterrupt: 

In [30]:
# parameters of the general model
topic_model_fitted.get_params()

{'calculate_probabilities': False,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': <bertopic.backend._sentencetransformers.SentenceTransformerBackend at 0x7fc9b2b8e890>,
 'hdbscan_model': HDBSCAN(min_cluster_size=10, prediction_data=True),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 1),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, low_memory=False, metric='cosine', min_dist=0.0, n_components=5, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(stop_words='english'),
 'verbose': False,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [31]:
# get info on topics (names, important words, representative reddit message/document)
topic_info = topic_model_fitted.get_topic_info()
topic_info.to_csv("topic_info_10000_method2.csv", sep="\t", index=False)

In [32]:
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4486,-1_like_im_people_think,"[like, im, people, think, right, time, thats, ...",[little hypocritical knowing Elon officially b...
1,0,325,0_congress_court_constitution_fbi,"[congress, court, constitution, fbi, doj, trum...",[actually read dred Scott case. case: case ini...
2,1,212,1_housing_roads_rent_city,"[housing, roads, rent, city, cities, property,...",[unless LARGE group left isolated large group ...
3,2,182,2_capitalism_socialism_communism_marx,"[capitalism, socialism, communism, marx, capit...","[Look, get post vague attempt (I don’t think d..."
4,3,137,3_ukraine_russia_russian_ukrainian,"[ukraine, russia, russian, ukrainian, nato, wa...",[appreciate these: cyber cat-and-mouse game en...
...,...,...,...,...,...
142,141,11,141_illusion_philosophy_negated_stiff,"[illusion, philosophy, negated, stiff, boils, ...","[Oh yeah, read philosophy scrutiny particularl..."
143,142,11,142_dt_libtard_silly_artemis,"[dt, libtard, silly, artemis, magma, boredom, ...",[u/MasterOfLords1 literally post DT instead go...
144,143,11,143_games_tv_directtv_slot,"[games, tv, directtv, slot, salt, lake, month,...",[Yeah would insane deal. Right NFL Sunday Tick...
145,144,10,144_algebra_progress_lbs_learn,"[algebra, progress, lbs, learn, arrogant, humb...",[Strategy? wrong you? strategy. think twenties...


In [33]:
# get the top words for each topic id (form the topic names)
topic_model_fitted.get_topics()

{-1: [('like', np.float64(0.005457766756503826)),
  ('im', np.float64(0.005241239288523216)),
  ('people', np.float64(0.005198990631854042)),
  ('think', np.float64(0.00497459671874603)),
  ('right', np.float64(0.004489606823968064)),
  ('time', np.float64(0.004434245286741447)),
  ('thats', np.float64(0.004235269518283284)),
  ('want', np.float64(0.00418942080252323)),
  ('say', np.float64(0.004176479405618831)),
  ('trump', np.float64(0.004133370648587436))],
 0: [('congress', np.float64(0.019824662583576104)),
  ('court', np.float64(0.01752158949664019)),
  ('constitution', np.float64(0.015682087287888634)),
  ('fbi', np.float64(0.015109914746788855)),
  ('doj', np.float64(0.013548522762800224)),
  ('trump', np.float64(0.012798855784095134)),
  ('law', np.float64(0.012492197507038914)),
  ('arms', np.float64(0.011484721234610456)),
  ('charges', np.float64(0.011005167451801825)),
  ('justice', np.float64(0.010344081427403054))],
 1: [('housing', np.float64(0.028965133719964332)),
  